## Purpose: Try different models-- Part3.
### Grid search with upsampling and scaling.

In [1]:
# import dependencies.
import pandas as pd
import numpy as np

from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

#### STEP1: Read in dataset.  Remove data from 2016-2019.
- data from 2016-2018 will be used to bs test the model.
- data from 2019 will be used to predict the winners of the 2019 WS.

In [2]:
# read in the data.
team_data = pd.read_csv("../../Resources/clean_data_1905.csv")
del team_data["Unnamed: 0"]
team_data.head()

,team,year,A,DP,E,G2,GS2,INN,PB,PO,...,R1,SHO,SO1,SV,TBF,W,WHIP,WP,WPCT,winners
0,St. Louis Cardinals,2019,1033,114,43,104,936,8313.0,3,2771,...,456,4,895,33,3896,56,1.29,21,0.538,0
1,Arizona Diamondbacks,2019,1010,83,45,105,945,8538.0,2,2846,...,472,7,925,24,4001,53,1.28,35,0.505,0
2,Kansas City Royals,2019,990,105,45,106,954,8421.0,6,2807,...,543,5,816,24,4125,39,1.46,34,0.368,0
3,Houston Astros,2019,875,54,50,106,954,8589.0,6,2863,...,432,7,1074,27,3929,67,1.14,31,0.632,0
4,Tampa Bay Rays,2019,975,92,53,107,963,8760.0,11,2920,...,409,6,1037,26,3985,59,1.16,40,0.551,0


In [3]:
# remove data from 2016 through 2019.
team_data_new = team_data.loc[team_data["year"] < 2016]
team_data_new.head()

,team,year,A,DP,E,G2,GS2,INN,PB,PO,...,R1,SHO,SO1,SV,TBF,W,WHIP,WP,WPCT,winners
120,San Francisco Giants,2015,1639,136,72,162,1458,13143.0,6,4381,...,631,11,1309,43,6048,87,1.21,40,0.537,0
121,Washington Nationals,2015,1425,142,73,162,1458,13137.0,17,4379,...,612,12,1476,46,6036,95,1.19,47,0.586,0
122,Houston Astros,2015,1599,135,77,162,1458,13212.0,18,4404,...,701,8,1396,44,6180,84,1.29,98,0.519,0
123,Detroit Tigers,2015,1537,148,75,161,1449,12852.0,5,4284,...,721,8,1232,47,6048,86,1.32,44,0.534,0
124,Boston Red Sox,2015,1427,139,75,162,1458,12957.0,37,4319,...,694,5,1362,43,6073,93,1.27,52,0.574,0


In [4]:
target = team_data_new["winners"]
features = team_data_new.drop({"team", "year", "winners"}, axis=1)
feature_columns = list(features.columns)
print (target.shape)
print (features.shape)
print (feature_columns)

(2344,)
(2344, 52)
['A', 'DP', 'E', 'G2', 'GS2', 'INN', 'PB', 'PO', 'TC', '2B', '3B', 'AB', 'AO', 'BB', 'G', 'H', 'HBP', 'HR', 'NP_x', 'OBP', 'OPS_x', 'PA', 'R', 'RBI', 'SAC', 'SB', 'SLG', 'TB', 'XBH', 'BB1', 'BK', 'CG', 'ER', 'ERA', 'G1', 'GF', 'GS', 'H1', 'HB', 'HR1', 'IP', 'L', 'OBP1', 'R1', 'SHO', 'SO1', 'SV', 'TBF', 'W', 'WHIP', 'WP', 'WPCT']


#### STEP2: Upsample and scale data.

In [5]:
# reset the index.
team_data_new = team_data_new.reset_index().drop({"index"}, axis=1)
team_data_new.head()

,team,year,A,DP,E,G2,GS2,INN,PB,PO,...,R1,SHO,SO1,SV,TBF,W,WHIP,WP,WPCT,winners
0,San Francisco Giants,2015,1639,136,72,162,1458,13143.0,6,4381,...,631,11,1309,43,6048,87,1.21,40,0.537,0
1,Washington Nationals,2015,1425,142,73,162,1458,13137.0,17,4379,...,612,12,1476,46,6036,95,1.19,47,0.586,0
2,Houston Astros,2015,1599,135,77,162,1458,13212.0,18,4404,...,701,8,1396,44,6180,84,1.29,98,0.519,0
3,Detroit Tigers,2015,1537,148,75,161,1449,12852.0,5,4284,...,721,8,1232,47,6048,86,1.32,44,0.534,0
4,Boston Red Sox,2015,1427,139,75,162,1458,12957.0,37,4319,...,694,5,1362,43,6073,93,1.27,52,0.574,0


In [6]:
# remove team and year.
feature_columns_new = feature_columns + ["winners"]
team_data_new = team_data[feature_columns_new]
team_data_new.head()

,A,DP,E,G2,GS2,INN,PB,PO,TC,2B,...,R1,SHO,SO1,SV,TBF,W,WHIP,WP,WPCT,winners
0,1033,114,43,104,936,8313.0,3,2771,3847,157,...,456,4,895,33,3896,56,1.29,21,0.538,0
1,1010,83,45,105,945,8538.0,2,2846,3901,203,...,472,7,925,24,4001,53,1.28,35,0.505,0
2,990,105,45,106,954,8421.0,6,2807,3842,185,...,543,5,816,24,4125,39,1.46,34,0.368,0
3,875,54,50,106,954,8589.0,6,2863,3788,200,...,432,7,1074,27,3929,67,1.14,31,0.632,0
4,975,92,53,107,963,8760.0,11,2920,3948,195,...,409,6,1037,26,3985,59,1.16,40,0.551,0


In [7]:
# upsample for a more balanced dataset.
def upsample(dataset, no_samples):
    '''
    INPUT: 
    -dataset = dataset without team names and year.
    -n_samples = number of minority_unsampled.
    
    OUTPUT:
    -X_train_scaled = scaled X train data.
    -X_test_scaled = scaled X test data.
    -y_train = y train data
    -y_test = y test data
    
    DESCRIPTION:
    -dataset is taken in and split into minority and majority classes.
    -dataset is then upsampled for the mainority class
    -split the data into features and targets
    -split data into train and test sets
    -train and test sets were are scaled.
    '''
    
    # separate majority and minority classes.
    df_majority = dataset.loc[dataset["winners"] == 0]
    df_minority = dataset.loc[dataset["winners"] == 1]

    # upsample minority class.
    df_minority_unsampled = resample(df_minority,
                                    replace=True,
                                    n_samples=no_samples,
                                    random_state=123)

    # combine majority class with upsampled minority class.
    df_upsampled = pd.concat([df_majority, df_minority_unsampled])

    # separate features and target.
    y = df_upsampled["winners"]
    X = df_upsampled[feature_columns]
    
    # split into train and test sets.
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    # scale X_train and X_test.
    scaler = StandardScaler()

    # transform the training and testing data.
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

In [8]:
# Do three different upsamplings.
X_train_100, X_test_100, y_train_100, y_test_100 = upsample(team_data_new, 2234)
X_train_50, X_test_50, y_train_50, y_test_50 = upsample(team_data_new, 1117)
X_train_25, X_test_25, y_train_25, y_test_25 = upsample(team_data_new, 559)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anacon

#### STEP3: Grid Search Model--Logistic Regression.

In [9]:
def grid_search_logistic(X_train, X_test, y_train, y_test):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -X_test = scaled X test data.
    -y_train = y train data.
    -y_test = y test data.
    
    OUTPUT:
    -classification report (has F1 score, precision and recall).
    -grid = saved model for prediction. 
    
    DESCRIPTION:
    -the scaled and split data is put through a grid search with logistic.
    -the model is trained.
    -a prediction is made.
    -print out the classification report and give the model.
    '''
    
    # fit the model.
    model = LogisticRegression(solver="lbfgs", max_iter= 2000)
    
    # create gridsearch estimator.
    param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}
    grid = GridSearchCV(model, param_grid, verbose=3)

    # fit the model.
    grid.fit(X_train, y_train)

    # predict.
    prediction = grid.predict(X_test)
    
    # print out the basic information about the grid search.
    print (grid.best_params_)
    print (grid.best_score_)
    print (grid.best_estimator_)
    
    grid = grid.best_estimator_
    predictions = grid.predict(X_test)
    print (classification_report(y_test, prediction, target_names=["0", "1"]))
    
    return grid

In [10]:
# for 1 part 0 to 1 part 1
model_100 = grid_search_logistic(X_train_100, X_test_100, y_train_100, y_test_100)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.7122929380993898, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.7390924956369983, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.7161572052401747, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7244986922406277, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7373472949389179, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7100436681222707, total=   0.0s
[CV] C=0.1 ...........................................................
[CV] ............

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.9s finished


{'C': 100}
0.7550901687027342
LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=2000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
              precision    recall  f1-score   support

           0       0.77      0.72      0.75       607
           1       0.71      0.76      0.73       540

   micro avg       0.74      0.74      0.74      1147
   macro avg       0.74      0.74      0.74      1147
weighted avg       0.74      0.74      0.74      1147



In [11]:
# for 1 part 0 to 0.5 part 1
model_50 = grid_search_logistic(X_train_50, X_test_50, y_train_50, y_test_50)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.7453917050691244, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................. C=0.001, score=0.726643598615917, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................. C=0.001, score=0.707852193995381, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7407834101382489, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7508650519031141, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7147806004618937, total=   0.0s
[CV] C=0.1 ...........................................................
[CV] ............

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................... C=1, score=0.7681660899653979, total=   0.1s
[CV] C=1 .............................................................
[CV] .................... C=1, score=0.7424942263279446, total=   0.1s
[CV] C=10 ............................................................
[CV] ................... C=10, score=0.7580645161290323, total=   0.1s
[CV] C=10 ............................................................
[CV] ................... C=10, score=0.7670126874279123, total=   0.1s
[CV] C=10 ............................................................
[CV] ................... C=10, score=0.7367205542725174, total=   0.1s
[CV] C=100 ...........................................................
[CV] ................................ C=100, score=0.75, total=   0.3s
[CV] C=100 ...........................................................
[CV] .................. C=100, score=0.7670126874279123, total=   0.2s
[CV] C=100 ...........................................................
[CV] .

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.4s finished


In [12]:
# for 1 part 0 to 0.25 part 1
model_25 = grid_search_logistic(X_train_25, X_test_25, y_train_25, y_test_25)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.8131868131868132, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.8118131868131868, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.8126721763085399, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.8228021978021978, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.8255494505494505, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.8278236914600551, total=   0.0s
[CV] C=0.1 ...........................................................
[CV] ............

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................... C=1, score=0.8241758241758241, total=   0.0s
[CV] C=1 .............................................................
[CV] .................... C=1, score=0.8269230769230769, total=   0.0s
[CV] C=1 .............................................................
[CV] .................... C=1, score=0.8278236914600551, total=   0.1s
[CV] C=10 ............................................................
[CV] ................... C=10, score=0.8241758241758241, total=   0.1s
[CV] C=10 ............................................................
[CV] ................... C=10, score=0.8337912087912088, total=   0.1s
[CV] C=10 ............................................................
[CV] ................... C=10, score=0.8278236914600551, total=   0.1s
[CV] C=100 ...........................................................
[CV] .................. C=100, score=0.8241758241758241, total=   0.2s
[CV] C=100 ...........................................................
[CV] .

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.2s finished


This is not significantly better than the straight logistic regression.

In [13]:
def predict_the_winner(model, year, team_data, X_train):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -model = the saved model.
    -team_data = complete dataframe with all data.
    -year = the year want to look at.
    
    OUTPUT:
    -printed prediction.
    
    DESCRIPTION:
    -data from year of interest is isolated.
    -the data are scaled.
    -the prediction is made.
    -print out the resulting probability and the name of the team.
    '''
    
    # grab the data.
    team_data = team_data.loc[team_data["year"] == year].reset_index()

    # set features (no team, year, winners).
    # set target (winners).
    features = team_data[feature_columns]
    
    # scale X_train and X_test.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    features = scaler.fit_transform(features)
    
    # fit the model.
    probabilities = model.predict_proba(features)

    # convert predictions to datafram.e
    WS_predictions = pd.DataFrame(probabilities[:,1])

    # Sort the DataFrame (descending)
    WS_predictions = WS_predictions.sort_values(0, ascending=False)

    WS_predictions['Probability'] = WS_predictions[0]

    # Print 50 highest probability HoF inductees from still eligible players
    for i, row in WS_predictions.head(50).iterrows():
       prob = ' '.join(('WS Probability =', str(row['Probability'])))
       print('')
       print(prob)
       print(team_data.iloc[i,1:27]["team"])

In [14]:
# predict for 2018.
predict_the_winner(model_100, 2018, team_data, X_train_100)


WS Probability = 0.999999964761129
Detroit Tigers

WS Probability = 0.9999996285630469
Cincinnati Reds

WS Probability = 0.9999850940697546
Washington Nationals

WS Probability = 0.9997832754224902
St. Louis Cardinals

WS Probability = 0.9996526771139598
Los Angeles Angels

WS Probability = 0.9985037515633309
Chicago White Sox

WS Probability = 0.9946075595123754
Houston Astros

WS Probability = 0.990330650797241
Los Angeles Dodgers

WS Probability = 0.9874756708813824
Minnesota Twins

WS Probability = 0.9662448074022902
Atlanta Braves

WS Probability = 0.9296811913766913
Miami Marlins

WS Probability = 0.9261972766826734
New York Yankees

WS Probability = 0.9080182393018879
Baltimore Orioles

WS Probability = 0.8983173120915562
Kansas City Royals

WS Probability = 0.8778620880891199
Arizona Diamondbacks

WS Probability = 0.5822933180378443
Toronto Blue Jays

WS Probability = 0.35424086211448425
Oakland Athletics

WS Probability = 0.1523456905990915
Chicago Cubs

WS Probability = 0.12

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [15]:
# predict for 2017.
predict_the_winner(model_100, 2017, team_data, X_train_100)


WS Probability = 0.9999999988486212
Colorado Rockies

WS Probability = 0.9999998879092996
Detroit Tigers

WS Probability = 0.9999998118923904
Milwaukee Brewers

WS Probability = 0.9999984405875323
Los Angeles Angels

WS Probability = 0.9999924276139065
Los Angeles Dodgers

WS Probability = 0.9999769223252329
Texas Rangers

WS Probability = 0.9998340738446169
Washington Nationals

WS Probability = 0.999180191887247
Kansas City Royals

WS Probability = 0.9991641163607011
Toronto Blue Jays

WS Probability = 0.9952583376260747
Houston Astros

WS Probability = 0.9859395682061786
Tampa Bay Rays

WS Probability = 0.9780089452205005
Cincinnati Reds

WS Probability = 0.9587338166476298
Chicago Cubs

WS Probability = 0.9087267881375077
Baltimore Orioles

WS Probability = 0.8532952660208495
Philadelphia Phillies

WS Probability = 0.8221110836162129
Minnesota Twins

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)




WS Probability = 0.7036960868670101
Cleveland Indians

WS Probability = 0.49449207468977024
New York Yankees

WS Probability = 0.11394842875743087
Chicago White Sox

WS Probability = 0.05536653934583901
Atlanta Braves

WS Probability = 0.018313131901649803
Boston Red Sox

WS Probability = 0.00932867870032541
Seattle Mariners

WS Probability = 0.000327208595226029
St. Louis Cardinals

WS Probability = 5.69989601704931e-05
New York Mets

WS Probability = 3.69296536204151e-05
Oakland Athletics

WS Probability = 2.313565384615397e-06
Arizona Diamondbacks

WS Probability = 5.659999788752788e-08
San Francisco Giants

WS Probability = 1.0342974629902376e-08
San Diego Padres

WS Probability = 3.940017347393901e-09
Pittsburgh Pirates

WS Probability = 4.541825192820079e-13
Miami Marlins


In [16]:
# predict for 2016.
predict_the_winner(model_100, 2016, team_data, X_train_100)


WS Probability = 0.9999999998735176
Cleveland Indians

WS Probability = 0.9999999968497826
Detroit Tigers

WS Probability = 0.9999996361519319
Minnesota Twins

WS Probability = 0.999969365516324
Cincinnati Reds

WS Probability = 0.9998233546814135
Oakland Athletics

WS Probability = 0.9994226861211745
New York Yankees

WS Probability = 0.9993336778114642
Chicago White Sox

WS Probability = 0.9993242189237599
Los Angeles Dodgers

WS Probability = 0.9962058531127479
Arizona Diamondbacks

WS Probability = 0.9954863960988767
New York Mets

WS Probability = 0.9877314055478
Houston Astros

WS Probability = 0.9777560958828747
Texas Rangers

WS Probability = 0.7743181215541584
Chicago Cubs

WS Probability = 0.7358695110597738
Colorado Rockies

WS Probability = 0.7179334470247518
Washington Nationals

WS Probability = 0.6000973211182504
Seattle Mariners

WS Probability = 0.46988155742522875
Los Angeles Angels

WS Probability = 0.1414879389151629
Pittsburgh Pirates

WS Probability = 0.047661279

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


#### STEP4: Grid Search Model--SVC.

In [17]:
def grid_search_svc(X_train, X_test, y_train, y_test):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -X_test = scaled X test data.
    -y_train = y train data.
    -y_test = y test data.
    
    OUTPUT:
    -classification report (has F1 score, precision and recall).
    -grid = saved model for prediction. 
    
    DESCRIPTION:
    -the scaled and split data is put through a grid search with svc.
    -the model is trained.
    -a prediction is made.
    -print out the classification report and give the model.
    '''
    
    # set up svc model.
    model = SVC(kernel="rbf", probability=True)

    # create gridsearch estimator.
    param_grid = {"C": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                 "gamma": [0.0001, 0.001, 0.01, 0.1]}
    grid = GridSearchCV(model, param_grid, verbose=3)

    # fit the model.
    grid.fit(X_train, y_train)

    # predict.
    prediction = grid.predict(X_test)
    
    # print out the basic information about the grid search.
    print (grid.best_params_)
    print (grid.best_score_)
    print (grid.best_estimator_)
    
    grid = grid.best_estimator_
    predictions = grid.predict(X_test)
    print (classification_report(y_test, prediction, target_names=["0", "1"]))
    
    return grid

In [18]:
# for 1 part 0 to 1 part 1
model_100 = grid_search_svc(X_train_100, X_test_100, y_train_100, y_test_100)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 28 candidates, totalling 84 fits
[CV] C=0.0001, gamma=0.0001 ..........................................
[CV] . C=0.0001, gamma=0.0001, score=0.5074106364428945, total=   2.2s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.5069808027923212, total=   2.1s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.9s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.5074235807860262, total=   2.1s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.5074106364428945, total=   2.2s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.5069808027923212, total=   2.3s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.5074235807860262, total=   2.2s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.5074106364428945, total=   2.2s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.5069808027923212, total=   2.3s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.5074235807860262, total=   2.2s
[CV] C=0.0001, gamma=0.1 .............................................
[CV] .

[CV] ..... C=10, gamma=0.0001, score=0.7401918047079338, total=   1.6s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.7382198952879581, total=   1.7s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.7152838427947599, total=   1.6s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7654751525719268, total=   1.4s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7923211169284468, total=   1.5s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7537117903930131, total=   1.4s
[CV] C=10, gamma=0.01 ................................................
[CV] ....... C=10, gamma=0.01, score=0.9084568439407149, total=   1.1s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:  2.8min finished


{'C': 10, 'gamma': 0.1}
0.9883653286794648
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       607
           1       0.99      1.00      0.99       540

   micro avg       0.99      0.99      0.99      1147
   macro avg       0.99      0.99      0.99      1147
weighted avg       0.99      0.99      0.99      1147



In [19]:
# for 1 part 0 to 0.5 part 1
model_50 = grid_search_svc(X_train_50, X_test_50, y_train_50, y_test_50)

Fitting 3 folds for each of 28 candidates, totalling 84 fits
[CV] C=0.0001, gamma=0.0001 ..........................................


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . C=0.0001, gamma=0.0001, score=0.6705069124423964, total=   0.8s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV] .. C=0.0001, gamma=0.0001, score=0.671280276816609, total=   0.8s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.6709006928406467, total=   0.8s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.6705069124423964, total=   0.9s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] ... C=0.0001, gamma=0.001, score=0.671280276816609, total=   0.8s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.6709006928406467, total=   0.8s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.6705069124423964, total=   0.8s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] .... C=0.0001, gamma=0.01, score=0.671280276816609, total=   0.8s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.6709006928406467, total=   0.8s
[CV] C=0.0001, gamma=0.1 .............................................
[CV] .

[CV] ..... C=10, gamma=0.0001, score=0.7534562211981567, total=   0.8s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ...... C=10, gamma=0.0001, score=0.740484429065744, total=   0.8s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.7251732101616628, total=   0.8s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7592165898617511, total=   0.8s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7785467128027682, total=   0.8s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7367205542725174, total=   0.8s
[CV] C=10, gamma=0.01 ................................................
[CV] ....... C=10, gamma=0.01, score=0.8559907834101382, total=   0.7s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:  1.3min finished


{'C': 10, 'gamma': 0.1}
0.9842368319876971
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       606
           1       0.98      1.00      0.99       261

   micro avg       0.99      0.99      0.99       867
   macro avg       0.99      1.00      0.99       867
weighted avg       0.99      0.99      0.99       867



In [20]:
# for 1 part 0 to 0.25 part 1
model_25 = grid_search_svc(X_train_25, X_test_25, y_train_25, y_test_25)

Fitting 3 folds for each of 28 candidates, totalling 84 fits
[CV] C=0.0001, gamma=0.0001 ..........................................


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . C=0.0001, gamma=0.0001, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.8126721763085399, total=   0.3s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.8126721763085399, total=   0.3s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.8118131868131868, total=   0.4s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.8126721763085399, total=   0.3s
[CV] C=0.0001, gamma=0.1 .............................................
[CV] .

[CV] ..... C=10, gamma=0.0001, score=0.8118131868131868, total=   0.4s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.8118131868131868, total=   0.4s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.8126721763085399, total=   0.4s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.8118131868131868, total=   0.4s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.8118131868131868, total=   0.4s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.8126721763085399, total=   0.4s
[CV] C=10, gamma=0.01 ................................................
[CV] ....... C=10, gamma=0.01, score=0.8791208791208791, total=   0.4s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:   38.1s finished


{'C': 10, 'gamma': 0.1}
0.9660861594867094
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       579
           1       0.95      0.91      0.93       149

   micro avg       0.97      0.97      0.97       728
   macro avg       0.96      0.95      0.96       728
weighted avg       0.97      0.97      0.97       728



Huh.  That's pretty good.  Try out model_100 and model_50 with the 2016-2018 stuff.

#### STEP5: Predict 2016-2018 winners with SVC Grid Search.

In [21]:
# predict for 2018.
predict_the_winner(model_100, 2018, team_data, X_train_100)


WS Probability = 4.3496838021795495e-07
Arizona Diamondbacks

WS Probability = 2.5307163074777594e-07
Boston Red Sox

WS Probability = 2.017251345442822e-07
Chicago Cubs

WS Probability = 1.9725634930693067e-07
Oakland Athletics

WS Probability = 1.8917422210265595e-07
St. Louis Cardinals

WS Probability = 1.4894491371606698e-07
Cleveland Indians

WS Probability = 1.4144377005208974e-07
Tampa Bay Rays

WS Probability = 1.3699951484977676e-07
New York Yankees

WS Probability = 1.3155018850613898e-07
Houston Astros

WS Probability = 1.298617346876442e-07
Texas Rangers

WS Probability = 1.2772874934941404e-07
Miami Marlins

WS Probability = 1.230592108659774e-07
Chicago White Sox

WS Probability = 1.1664091087923763e-07
Detroit Tigers

WS Probability = 1.150221205132816e-07
Cincinnati Reds

WS Probability = 1.1142946497045233e-07
Los Angeles Dodgers

WS Probability = 1.0623098130561852e-07
Seattle Mariners

WS Probability = 1.0619917718396513e-07
Minnesota Twins

WS Probability = 1.01540

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [22]:
# predict for 2017.
predict_the_winner(model_100, 2017, team_data, X_train_100)


WS Probability = 2.974661164068443e-07
Los Angeles Angels

WS Probability = 1.7328883400192622e-07
Houston Astros

WS Probability = 1.521698657711041e-07
Los Angeles Dodgers

WS Probability = 1.45144590097298e-07
Tampa Bay Rays

WS Probability = 1.450319127526764e-07
Washington Nationals

WS Probability = 1.3586354422200113e-07
Milwaukee Brewers

WS Probability = 1.2070498914074586e-07
Boston Red Sox

WS Probability = 1.2032711231975875e-07
Chicago Cubs

WS Probability = 1.1996606770539417e-07
Kansas City Royals

WS Probability = 1.1715344534736105e-07
Miami Marlins

WS Probability = 1.1564147200412902e-07
Cleveland Indians

WS Probability = 1.152307431249863e-07
Oakland Athletics

WS Probability = 1.0693786234346138e-07
Chicago White Sox

WS Probability = 1.0586702292048753e-07
Pittsburgh Pirates

WS Probability = 1.0000000994736041e-07
San Diego Padres

WS Probability = 1.0000000994736041e-07
Baltimore Orioles

WS Probability = 1.0000000994736041e-07
Texas Rangers

WS Probability = 

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [23]:
# predict for 2016
predict_the_winner(model_100, 2016, team_data, X_train_100)


WS Probability = 2.083070311618444e-07
Oakland Athletics

WS Probability = 1.812658687541172e-07
Colorado Rockies

WS Probability = 1.5759737889264882e-07
Minnesota Twins

WS Probability = 1.3029146405162224e-07
Washington Nationals

WS Probability = 1.285148494027263e-07
New York Yankees

WS Probability = 1.1908673377823504e-07
Boston Red Sox

WS Probability = 1.1758958890442776e-07
Los Angeles Dodgers

WS Probability = 1.1737675477072851e-07
Cleveland Indians

WS Probability = 1.1324538013034387e-07
Chicago Cubs

WS Probability = 1.1151381240719608e-07
San Diego Padres

WS Probability = 1.1121014913387123e-07
Houston Astros

WS Probability = 1.0000000994736041e-07
Milwaukee Brewers

WS Probability = 1.0000000994736041e-07
Texas Rangers

WS Probability = 1.0000000994736041e-07
Pittsburgh Pirates

WS Probability = 1.0000000994736041e-07
Chicago White Sox

WS Probability = 1.0000000994736041e-07
Seattle Mariners

WS Probability = 1.0000000994736041e-07
Tampa Bay Rays

WS Probability = 

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [24]:
# predict for 2018
predict_the_winner(model_50, 2018, team_data, X_train_50)


WS Probability = 0.00012157267912213627
Arizona Diamondbacks

WS Probability = 0.0001080795945730767
Boston Red Sox

WS Probability = 0.0001009163023827148
Cleveland Indians

WS Probability = 8.828478877467394e-05
Chicago Cubs

WS Probability = 8.342103322433089e-05
St. Louis Cardinals

WS Probability = 8.22424800664411e-05
Tampa Bay Rays

WS Probability = 7.716786524251512e-05
Texas Rangers

WS Probability = 7.367194116256595e-05
Miami Marlins

WS Probability = 7.335624302189575e-05
Chicago White Sox

WS Probability = 7.116230278548426e-05
Oakland Athletics

WS Probability = 7.095822411966283e-05
Detroit Tigers

WS Probability = 7.074222366904107e-05
Washington Nationals

WS Probability = 6.986363254751101e-05
Houston Astros

WS Probability = 6.872930019202327e-05
Cincinnati Reds

WS Probability = 6.755457529499457e-05
Seattle Mariners

WS Probability = 6.722210352867091e-05
Los Angeles Dodgers

WS Probability = 5.994115876802605e-05
Kansas City Royals

WS Probability = 5.97052821689

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [25]:
# predict for 2017
predict_the_winner(model_50, 2017, team_data, X_train_50)


WS Probability = 0.00017792423706647237
Los Angeles Angels

WS Probability = 0.00012141301367724824
Washington Nationals

WS Probability = 0.00010307741411686992
Tampa Bay Rays

WS Probability = 9.932998533280211e-05
Houston Astros

WS Probability = 8.686819725984906e-05
Milwaukee Brewers

WS Probability = 8.438210621931605e-05
Los Angeles Dodgers

WS Probability = 7.251505037703613e-05
Chicago White Sox

WS Probability = 7.196684682393447e-05
Cleveland Indians

WS Probability = 7.186129901382756e-05
Chicago Cubs

WS Probability = 7.131926853792886e-05
Boston Red Sox

WS Probability = 7.086832709543979e-05
Miami Marlins

WS Probability = 7.048309839874454e-05
Kansas City Royals

WS Probability = 6.725218249728795e-05
Pittsburgh Pirates

WS Probability = 6.703741301736968e-05
Oakland Athletics

WS Probability = 6.509105485401746e-05
Baltimore Orioles

WS Probability = 5.907887606221612e-05
Atlanta Braves

WS Probability = 5.6940262170687876e-05
New York Yankees

WS Probability = 5.5798

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [26]:
# predict for 2016
predict_the_winner(model_50, 2016, team_data, X_train_50)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)



WS Probability = 0.00014632159754961657
Oakland Athletics

WS Probability = 8.849413589062698e-05
Washington Nationals

WS Probability = 7.837820638039918e-05
Minnesota Twins

WS Probability = 7.639040834730378e-05
New York Yankees

WS Probability = 7.209392764493316e-05
Boston Red Sox

WS Probability = 7.209270753707973e-05
Cleveland Indians

WS Probability = 7.207317914209216e-05
Los Angeles Dodgers

WS Probability = 7.109016584676757e-05
Seattle Mariners

WS Probability = 6.969310555915081e-05
San Diego Padres

WS Probability = 6.598498353375206e-05
Houston Astros

WS Probability = 6.571973419355133e-05
Colorado Rockies

WS Probability = 6.032558202020395e-05
Detroit Tigers

WS Probability = 5.9513374027175637e-05
San Francisco Giants

WS Probability = 5.607906696395179e-05
Chicago White Sox

WS Probability = 4.9173164952514625e-05
Baltimore Orioles

WS Probability = 4.8030145552202655e-05
Cincinnati Reds

WS Probability = 4.358382690101108e-05
Chicago Cubs

WS Probability = 3.9493

This is not better.  The logistic regression without grid and upsampling works better than either upsampled thing.